# Protein Modeling and Design in Rosetta
Now that we are familiar with degrees of freedom in proteins, we will explore how we can manipulate these degrees of freedom to serve our design purposes. This lab will introduce some of the core modeling and design algorithms implemented in the [Rosetta Molecular Modeling Suite](https://rosettacommons.org/).

By the end of this tutorial, you will be familiar with:
* Computational representation of a protein
* Energetic evalutation of a protein conformation
* Protein conformational sampling algorithms

## Starting the PyMOLMover
Navigate to where you've installed PyRosetta on your computer and find the file called `PyMOL-RosettaServer.py`. The path should be `<Base path>/PyRosetta/src/PyMOL-RosettaServer.py`. Open up PyMOL and enter the command `run path/to/PyMOL-RosettaServer.py`. This will open a connection between Rosetta and PyMOL so that they can communicate.

This is what it should look like:

<img src="Assets/Tutorial_2/RunPyMOLServer.png" width="600" align="center"/>


Once the connection is established, we can pass a protein structure to PyMOL through Rosetta.

To run the following code, select the cell by clicking on it then click the "Run" button in the top bar. Alternatively, you can use `Ctrl + Enter` to run code in a selected cell or `Shift + Enter` run code in the selected cell and automatically move on to the next cell.

In [9]:
# Package and module imports
import os
import sys

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('seaborn-white')

import pyrosetta
import pyrosetta.rosetta as rosetta
from pyrosetta import toolbox

from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Label
import ipywidgets as widgets
from IPython.display import display

# PyRosetta init
pyrosetta.init('-ex1 -ex2 -extrachi_cutoff 0 -mute all')
sfxn = rosetta.core.scoring.get_score_function()
pymover = rosetta.protocols.moves.PyMOLMover()

# Widget init
output = widgets.Output()

PyRosetta-4 2019 [Rosetta devel 2019.40.post.dev+40.master.aeda51ea830d1d315912d2563a754cb38d9e49b8 2019-10-07T17:21:58] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


## Pose

In Rosetta, state of a particular protein is represented and stored in an object caled a Pose. We have been using the Pose to pass information to PyMOL, so we are already familiar with some of the information that the Pose contains.

The first major role of the Pose is to store information on the current **conformation** of the protein. The residue is the individual unit of a Pose, where each residue stores the degrees of freedom within ($\chi$ angles) and between ($\phi$,$\psi$,$\omega$) residues. It is important to note that Rosetta internally represents proteins using torsion angles since many modeling and design algorithms depend on manipulating these DOFs. The Pose is reponsible for converting these torsion angles into the xyz coordinates that we are familiar with. (Rosetta uses an algorithm called [Natural Extension Reference Frame](https://www.ncbi.nlm.nih.gov/pubmed/15898109) aka NeRF to convert between torsion and cartesian space.

The Pose also stores the **energy** of different components of protein. We will explore this in the next section. An in-depth discussion of the design decisions in Rosetta pertaining to the Pose can be found [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4083816/#S7title).

First, run the following cell to explore the information stored in a Pose. We will be working with PDB ID 1QYS, also known as Top7. [Kuhlman et al.](https://www.doi.org/10.1126/science.1089427) applied Rosetta to design the first *de novo* protein with a fold and topology never before observed in nature. 

In [2]:
# Get Information from a Pose
top7_pose = rosetta.core.import_pose.pose_from_file(os.path.join('Assets', 'Tutorial_3', '1qys.pdb.gz'))
pymover.apply(top7_pose)

def display_pose_info(position):
    current_residue = top7_pose.residue(position)
    print(current_residue)
    print(f'Mainchain torsions: {" ".join([str(a) for a in current_residue.mainchain_torsions()])}')
    print(f'Chi angles: {" ".join([str(a) for a in current_residue.chi()])}')

# Widgets
position_slider = widgets.SelectionSlider(options=range(1, top7_pose.size() + 1), description='Position')
display(interactive(display_pose_info, position=position_slider))

interactive(children=(SelectionSlider(description='Position', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

## Energy Function

In computational modeling of proteins, an energy function (aka score function, force field) is used to evaluate the potential energy of a particular conformation. For most protein engineering applications, the goal is to minimize the potential energy of a protein sequence/conformation while fulfilling design constraints (e.g. adopting a particular structure, creating a protein-protein interface).

A comprehensive treatment of the Rosetta energy function is outlined in [this paper](https://pubs.acs.org/doi/10.1021/acs.jctc.7b00125). It is *highly recommended* that you at least read through the introduction of this paper. We will unpack the highlights of the paper in this tutorial. In short, Rosetta's energy function is composed of indiviual terms that are evaluated for a particular residue (one-body term) or between residue pairs (two-body term). Weights are applied to each of these terms and summed for all residues and residue pairs to evaluate a total energy for a protein conformation. These weights are found through a process called parameterization, where the weights (adjustable parameters of the energy function) are adjusted in an iterative manner to find values that best balance contributions from individual score terms to recapitulate observed macromolecular structural data. This typcially means that the energy function evaluates a "real" structure as lower in energy compared to "decoy" structures. The process Rosetta uses to parameterize its energy function is outlined [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5515585/).

While many components of Rosetta's energy function use traditional units (e.g. kcal/mol), statistical terms and parameterization of Rosetta energy function results in Rosetta calculating energies with arbirary units (colloquially referred to as Rosetta Energy Units, REU). Being said, the latest iteration of Rosetta's energy function (REF2015) has been parameterized to better approximate units of kcal/mol.

The components to Rosetta's energy function are composed of physical terms you should already be familiar with (Lennard-Jones potential, Coulomb's law) as well as statistical terms that describe observed distubutions in protein structures (rotamer propensity, hydrogen bond geometries, Ramachandran space).

### Score Terms, Distilled

The purpose of this section is to distill each of the important terms in Rosetta's energy function into easily digestible bite-sized bits. Refer to [Alford et al.](https://pubs.acs.org/doi/10.1021/acs.jctc.7b00125) for more in-depth discussion of each of these terms, plus additional terms not covered here.

#### Van der Waals and Electrostatics

* **fa_atr:** Evaluates the attractive component of the Lennard-Jones 6-12 potential for Van der Waals interactions between residues.
* **fa_rep:** Evaluates the repulsive component of the Lennard-Jones 6-12 potential for Van der Waals interactions between residues.
* **fa_elec:** Evaluates the interaction energy between charged atoms.

The following figure illustrates how fa_atr, fa_rep, and fa_elec are evaluated for a pair of atoms with given atomic radii and charges. Note that the functions that Rosetta evaluates for these terms (blue) differs from the functions that you may be familiar with from your chemistry/physics courses (red). This is done for several reasons:
    
1. To allow different weightings of individual terms during modeling/design protocols. For instance, several commonly used Rosetta design protocols ramp fa_rep while keeping fa_atr constant to "relax" structures.
2. To apply heuristics that better reflect modeling of biological molecules.
3. To allow for easier/faster evaluation of energy terms. The functional forms of many score terms are altered to removed difficult-to-evaluate gradients or impossible-to-evaluate discontinuities.

<img src="Assets/Tutorial_3/Alford-farep_faatr.jpg" width="800" align="center"/>

#### Hydrogen Bonds

* **hbond_sc and hbond_bb_sc:** Evaluates the energy of side-chain-side-chain and backbone-side-chain hydrogen bonds, respectively. While the fa_elec term partially contributes to hydrogen bonds, hbond_sc and hbond_bb_sc specifically evaluates energies based on the observed hydrogen bonding geometries from high-resolution crystal structures. A hydrogen bond is composed of a donor atom, the donor atom's hydrogen, the hydrogen bond acceptor atom, and the acceptor atom base (Panel A). The hydridization of the acceptor atom affects the distribution of favorable hydrogen bond geometries depending on whether it is *sp$_2$*(Panel B) or *sp$\_3$* hybridized.

<img src="Assets/Tutorial_3/Alford-hbond_sc.jpg" width="800" align="center"/>

#### Solvation

* **fa_sol and lk_ball_wtd:** This pair of score terms evaluates the isotropic and anisotropic energy of desolvation between pairs of residues. You may have noticed that the structures we pass from Rosetta to PyMol do not include any waters. Instead of explicitly modeling waters in and around protein structures (as in an explicit solvation model), Rosetta opts for an implicit solvation model where the energetic contributions of water are approximated. Rosetta uses an implicit solvation model, specifically the [Lazaridis-Karplus Gaussian Exclusion Model](https://www.ncbi.nlm.nih.gov/pubmed/10753811), because explicitly modleing waters is computationally expensive and slow. For a pair of residues, this model calcualtes the energy that is required for atoms in residue A to desolvate atoms in residue B and visa-versa. Solvation in Rosetta is separated into fa_sol (panel A), an isotropic term that assumes water is evenly distributed around an atom, and lk_ball_wtd (panel B), an anisotropic term that accounts for ordered waters that may form hydrogen bonds with polar atoms. 

<img src="Assets/Tutorial_3/Alford-fa_sol.jpg" width="800" align="center"/>

#### Side chain Conformations

* **fa_dun:** Relates the probability that a side chain will adopt a rotameric conformation given the backbone $\phi$/$\psi$ torsion angles and residue identity into an energy with arbitrary units. Recall in tutorial 2 how backbone 
$\phi$/$\psi$ angles influence the probability observing discrete rotamer states (Panel A).

<img src="Assets/Tutorial_3/Alford-dunbrack.jpg" width="400" align="center"/>

#### Backbone-dependent terms

* **rama_prepro:** Relates the probability that the a position's $\phi$/$\psi$ torsions are observed given the current residue identity to an energy. This probability is affected by whether a proline is present (Panel B) or is not present (Panel C) at the next position in the protein sequence.

* **p_aa_pp:** Relates the probability that the current residue identity is observed given a position's $\phi$/$\psi$ torsions to an energy (Panel D). This score term is the inverse of rama_prepro.

<img src="Assets/Tutorial_3/Alford-rama.jpg" width="800" align="center"/>

#### Reference Energy

* **ref:** This is a score term that is specifically used for design. For each residue type, a reference energy is calculated that approximates the residue's contribution toward the free energy of unfolding.

Refer to Table 1 in [Alford et al.](https://pubs.acs.org/doi/10.1021/acs.jctc.7b00125) for brief descriptions of additional score terms with their corresponding weights and units.

### Interaction Graph

All energy terms that Rosetta calculates are stored in a stucture called a graph, where nodes represent residues and edges represent the interactions between them. Two-body energies that depend on two residues such as fa_rep and fa_elec are stored on edges, while one-body terms that only depend on a single residue such as fa_dun and rama_prepro are stored on nodes.

<img src="Assets/Tutorial_3/1QYS_IG_Annotated.png" width="800" align="center"/>

You may have noticed from the figures above that energies are only evaluated out to 6$\unicode{x212B}$. This is intentional: most score terms in Rosetta's energy function are only evaluated out to 6$\unicode{x212B}$ since interactions past that are assumed to be negligible. There is also a practical reason for this: as we alter conformations in the Pose (e.g. make a mutation, adjust a secondary structure element), we only need to recalulate energies in the interaction graph within 6$\unicode{x212B}$ of the change. This lets us keep previously calculated energies and reduces the number of energies we need to update. This is extremely important for design tasks, where a protocol might be making millions of small changes to a Pose!

Run the following cell to visualize the contributions of different score terms in the Pose of Top7. Select an energy from the dropdown menu and the energies will be painted onto the Pose, where blue represents low (good) energies and red represents high (bad) energies. For each score term, where are the problematic residues? Why?

In [3]:
# Visualize energies in a protein
def display_term_energy(score_term):
    pymover.label_energy(top7_pose, score_term)
    
# Interactive bits
score_terms = ['fa_atr',
               'fa_rep', 
               'fa_sol',
               'lk_ball_wtd',
               'fa_elec', 
               'hbond_sc', 
               'hbond_bb_sc',
               'fa_dun', 
               'rama_prepro',
               'p_aa_pp']

term_selection = widgets.Dropdown( options=score_terms, value='fa_rep', description='Score Term:')
display(interactive(display_term_energy, score_term=term_selection))

interactive(children=(Dropdown(description='Score Term:', index=1, options=('fa_atr', 'fa_rep', 'fa_sol', 'lk_…

# Manipulating the Pose

In the previous tutorial, we learned about the degrees of freedom proteins possess for us to manipulate. In the beginning of this tutorial, we learned how these degrees of freedom are stored in a Pose and how we can energetically evaluate a conformational state of the Pose with Rosetta's energy function. In this section, we will explore how we can combine these to perform modeling and design of proteins. The following algorithms are the workhorse protocols for searching protein sequence/conformational space for low-energy states.

## Packing
[The Packer](https://doi.org/10.1073/pnas.97.19.10383) is the machinery with which Rosetta samples side chain identities and conformations for a fixed backbone protein. Given a discrete set of side chain rotamers for a set of positions, the Packer samples possible states that minimize the overall energy of the Pose. Take the following backbone, where the yellow spheres represent positions where we want to apply packing. 

<br>
<img src="Assets/Tutorial_3/Scaffold-Apo.png" width="300" align="center"/>
<br>

For each position in the Pose where we want to sample side chain identities/conformations, a set of rotamers is generated for each residue identity using the $\phi$/$\psi$ torsions bins and rotamer probabilities in the [Dunbrack Rotatmer Library](https://doi.org/10.1016/j.str.2011.03.019). The rotamer set can be composed of residues of the same identity as the original residue (packing, protein sequence *does not* change as rotamers are sampled) or contain many residue types (design, protein sequence *does* change as rotamers are sampled).

<br>
<img src="Assets/Tutorial_3/RotamerSets.png" width="600" align="center"/>
<br>

### Simulated Annealing
Once these rotamer sets are generated, the Packer uses an algorithm called [Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing) to efficiently sample our search space (i.e. all possible combinations of rotamers in our rotamer set built into our fixed backbone scaffold). Simulated annealing proceeds one step at a time, where each step selects a random position, samples a rotamer from that position's RotamerSet, and evaluates whether the change should be kept in the context of the entire protein. The goal of simulated annealing is to find a rotamer combinations that yield low-energy solutions. This follows the assumption that proteins typically adopt the lowest-energy conformations available to them based on their sequence.

<br>
<img src="Assets/Tutorial_3/RotamerTrial.png" width="800" align="center"/>
<br>

The simulated annealing protocol is outlined in this flowchart:

<br>
<img src="Assets/Tutorial_3/PackerFlowchart.png" width="400" align="center"/>
<br>

In a typical packing run, ten geometrically decreasing temperatures are used for the outer loop, while the number of trials for the inner loop scales with number of rotamers in the set of rotamers.

The packer uses a rule called the Metropolis Criterion to decide whether or not to accept a new rotamer. For any update where the change in Pose energy $\Delta E = E_{current state} - E_{previous state}$:

* If $\Delta E < 0$, accept the move.
* If $\Delta E > 0$, calcualte the probability $p = e^{-\Delta E / T}$ and select a random number $n \in [0,1)$. If $p$ > $n$, accept the move. If $p$ < $n$, reject the move.

In the following example (source: [Wikipedia](https://en.wikipedia.org/wiki/Simulated_annealing#/media/File:Hill_Climbing_with_Simulated_Annealing.gif)), the simulated annealing algorithm is used to find the maximum value of a 2D function. Notice how most positions are accepted at high temperatures. As the temperature decreases, it is less likely that a new position will be accepted if the new value is a lower than the previously found value. As the temperature approaches zero, we converge upon the maximum value of the function.

<br>
<img src="Assets/Tutorial_3/Hill_Climbing_with_Simulated_Annealing.gif" width="500" align="center"/>
<br>

It is important to note that simulated annealing is a stochasitc protocol: since it uses randomness, it will potentially give you a different answer every time the protocol is run. It is also important to note that simulated annealing is not guarenteed to provide an optimal solution; simulated annealing is simply a method for sampling *good* solutions. Since protein design space is huge, we typically attempt thousands of simulated annealing trajectories for a single design problem to generate an ensemble of low-energy solutions.

### Repack the Core of Top7
Let's explore how to use the Packer to repack and design the core of Top7. First, run the following cell to run the Packer to explore how Rosetta will repack the existing core side-chains. Remember, repacking will not change the identities of the side-chains; the Packer will simply sample different rotamers of the existing residue to attempt to find low-energy conformations.

In [4]:
# Repack core of Top7
def apply_packer_to_pose(pose, pack_only=False):
    packer_pose = pose.clone()
    task_factory = rosetta.core.pack.task.TaskFactory()
    # rosetta.protocols.moves.AddPyMOLObserver(test_pose, True)

    positions = '2,4,6,8,21,27,31,34,35,38,46,48,50,61,65,69,74,86,88'
    position_rs = rosetta.core.select.residue_selector.ResidueIndexSelector(positions)
    natro_rs = rosetta.core.select.residue_selector.NotResidueSelector(position_rs)
    natro_op = rosetta.core.pack.task.operation.OperateOnResidueSubset(rosetta.core.pack.task.operation.PreventRepackingRLT(), natro_rs)
    task_factory.push_back(natro_op)
    
    if pack_only:
        natro_op = rosetta.core.pack.task.operation.OperateOnResidueSubset(rosetta.core.pack.task.operation.RestrictToRepackingRLT() , position_rs)
        task_factory.push_back(natro_op)

    packer_task = task_factory.create_task_and_apply_taskoperations(pose)
    packer_task.or_linmem_ig(True)  # Linear memory Interaction Graph

    sfxn(packer_pose)
    run_packer = rosetta.protocols.minimization_packing.PackRotamersMover(sfxn, packer_task)
    run_packer.apply(packer_pose)
    print(packer_pose)
    pymover.apply(packer_pose)
    
    return packer_pose
    
packed_top7_pose = apply_packer_to_pose(top7_pose, pack_only=True)

PDB file name: Assets/Tutorial_3/1qys.pdb.gz
Total residues: 92
Sequence: DIQVQVNIDDNGKNFDYTYTVTTESELQKVLNELMDYIKKQGAKRVRISITARTKKEAEKFAAILIKVFAELGYNDINVTFDGDTVTVEGQL
Fold tree:
FOLD_TREE  EDGE 1 92 -1 


### Redesign the Core of Top7
Now, let's try using the Packer to redesign the core of Top7. The Packer will attempt to build rotamers for different residue identities at each position in the core of Top7 to find low-energy conformations.

In [5]:
# Design core of Top7
designed_top7_pose = apply_packer_to_pose(top7_pose, pack_only=False)

PDB file name: Assets/Tutorial_3/1qys.pdb.gz
Total residues: 92
Sequence: DIQVQVNIDDNGKNFDYTYTVTTESEYQKVRNEVMDYTKKQGAKRVRVSVTARTKKEAEKFAAILIKVHAELGFNDINVTFDGDTCTVEGQL
Fold tree:
FOLD_TREE  EDGE 1 92 -1 


## Minimization

The Packer allows us to find low-energy conformations in sequence space by subsituting side-chain rotamers on a fixed-backbone structure. These conformations are made up of discrete rotamers and a fixed backbone; we sacrifice accurate representation of conformational space in exchange for sampling efficiency. We can apply minimization to find the *local minimum* of our energy function for a given conformation by making small adjustments to all degrees of freedom in a the Pose.

Minimization, or gradient descent, works by evaulating the gradient of the energy function with respect to the degrees of freedom (we will be looking at torsional DOFs here) in the backbone and side-chains of the protein. An important property of Rosetta's energy function is that we can take the derivative of each score term to evaluate the gradient of the full energy function. The gradient tells us which direction we should move (i.e. how to adjust each degree of freedom) to minimize the value of the overall energy function. By iteratively evaluating the gradient and making small moves in the direction of the gradient, we will converge on the local energy minima for the current conformation.

This video by [3Blue1Brown on YouTube](https://youtu.be/IHZwWFHWa-w?t=351) provides an excellent visual explanation behind the gradient decent algorithm (watch 5:51 - 8:34). In the context of the video, our cost funciton is the energy function and our input space is the current conformation of the pose.

Let's apply minimization to the redesigned Top7 core to see what adjustments will be made to find the local minimum of the energy function.

In [7]:
# Real-time minimization
minimized_redesigned_top7_pose = designed_top7_pose.clone()

# MoveMap Factory Settings
positions = '2,4,6,8,21,27,31,34,35,38,46,48,50,61,65,69,74,86,88'
position_rs = rosetta.core.select.residue_selector.ResidueIndexSelector(positions)
mm_enable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_enable
mm_disable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_disable
pose_movemapfactory = rosetta.core.select.movemap.MoveMapFactory()
pose_movemapfactory.add_bb_action(mm_enable, position_rs)
pose_movemapfactory.add_chi_action(mm_enable, position_rs)
pose_movemap = pose_movemapfactory.create_movemap_from_pose(minimized_redesigned_top7_pose)

minmover = pyrosetta.rosetta.protocols.minimization_packing.MinMover()
minmover.movemap(pose_movemap)
minmover.apply(minimized_redesigned_top7_pose)

## Designing a Protein De Novo


In [10]:
# Build a full protein de novo
sys.path.append(os.path.join(os.getcwd(),'Assets'))
sys.path.append(os.path.join(os.getcwd(),'Assets/Tutorial_2'))
from Assets.Tutorial_2.simple_pose_moves import insert_alas

# Function to remodel loops
def model_loops(pose, loop_start):
    loopmodeler_xml = f"""
    <MOVERS>
    <LoopModeler name="model_loop" config="kic" fast="True" >
        <Loop start="{loop_start-4}" stop="{loop_start + 8}" cut="{loop_start + 4}"/>
    </LoopModeler>
    </MOVERS>
    """
    rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string(loopmodeler_xml).get_mover("model_loop").apply(pose)
    
denovo = rosetta.core.pose.Pose()
make_bundle_xml = f'''
<SCOREFXNS>
    <ScoreFunction name="sfxn" weights="ref2015"/>
</SCOREFXNS>
<MOVERS>
    <MakeBundle name="denovo" helix_length="30" omega0="0.08" crick_params_file="alpha_helix" symmetry="2" r0="8">
        <Helix delta_omega1="1.57"/>
        <Helix invert="1" delta_omega0="1.5707963" delta_omega1="1.57" z0_offset="3.14"/>
    </MakeBundle>
</MOVERS>'''
rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string(make_bundle_xml).get_mover("denovo").apply(denovo)

# Add four residues to helicies for connections
for loop_start in [30, 64, 98]:
    insert_alas(denovo, loop_start, 4)
    model_loops(denovo, loop_start)

# Create backbone
pymover.apply(denovo)

# Design sequence onto backbone
# todo: modify apply_packer_to_pose() to optionally take natro/designable positions
apply_packer_to_pose(denovo)

# Remodel loops after adding sequence
for loop_start in [30, 64, 98]:
    model_loops(denovo, loop_start)
    
# Design sequence onto backbone
apply_packer_to_pose(denovo)

pose_movemapfactory = rosetta.core.select.movemap.MoveMapFactory()
pose_movemapfactory.add_bb_action(mm_enable, position_rs)
pose_movemapfactory.add_chi_action(mm_enable, position_rs)
pose_movemap = pose_movemapfactory.create_movemap_from_pose(minimized_redesigned_top7_pose)

minmover = pyrosetta.rosetta.protocols.minimization_packing.MinMover()
minmover.movemap(pose_movemap)
minmover.apply(minimized_redesigned_top7_pose)

Total residues: 132
Sequence: AGAEAAAWAAAAAAAAAAAALAAAAADAAASAAEDAAAAAAAAAADAKARAAAAAAAAAANAAAYAAASAAAAAAAAAAAAAAAAKALAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Fold tree:
FOLD_TREE  EDGE 1 132 1  EDGE 132 103 -1  EDGE 1 102 -1 
Total residues: 132
Sequence: AGAEAAAKAAAAAAAAAAAALAAAAASAAALAAVDAAAAAAAAAADAKARAAAAAAAAAANAAAEAAASAAAAAAAAAAAAAAAAKALAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Fold tree:
FOLD_TREE  EDGE 1 132 1  EDGE 132 103 -1  EDGE 1 102 -1 


## Direct Applications of De Novo Protein Design
[Massively parallel de novo protein design for targeted therapeutics](https://www.ncbi.nlm.nih.gov/pubmed/28953867)
Antibody loop design
Biosensors
Enzyme design

## A Note on More Complex Structure Prediction and Design Strategies

Here we only do a single simulation to familiarize ourselvex with the core concepts behind these methods
The methods covered in this tutorial are typically run on HPC clusters with 10^4-10^6 individual trajectories

More complex protocols require resources outside the scope of an introductory course
Homology Modeling
Ab initio
Fragment-based Loop Modeling
Require large databases, many trajectories for meaningful and interpretable data

## Deliverables

* Submit your repacked, redesigned, and minimized versions of Top7. Explain the decisions Rosetta made and why it may have made those decisions. What went wrong? Provide a hypothesis for why and how you would fix these issues. 